In [287]:
import os
import pandas as pd
import re
import random

In [288]:
r1_hs_seqs_path = '/Users/stephaniecrilly/Library/CloudStorage/Box-Box/kortemmelab/home/scrilly/helix_sliding/20231212_final_analysis_r1_order/20231221_final_order/20231221_all_design_sets_r1_order.csv'
r2_ssd_seqs_path = '/Users/stephaniecrilly/Library/CloudStorage/Box-Box/kortemmelab/home/scrilly/helix_sliding/20250604_r2_hs_lib/metric_files/ssd/ssd_designs_passing.csv'
r2_msd_seqs_path = '/Users/stephaniecrilly/Library/CloudStorage/Box-Box/kortemmelab/home/scrilly/helix_sliding/20250604_r2_hs_lib/metric_files/msd/msd_designs_passing.csv'
r2_msd_exp_bbs_seqs_path = '/Users/stephaniecrilly/Library/CloudStorage/Box-Box/kortemmelab/home/scrilly/helix_sliding/20250604_r2_hs_lib/metric_files/msd_exp_bbs/exp_bbs_msd_designs_passing.csv'
r2_msd_scale_up_seqs_path = '/Users/stephaniecrilly/Library/CloudStorage/Box-Box/kortemmelab/home/scrilly/helix_sliding/20250604_r2_hs_lib/metric_files/msd_f2s_os_scaled_up/outputs/msd_designs_passing.csv'

outdir = '/Users/stephaniecrilly/Library/CloudStorage/Box-Box/kortemmelab/home/scrilly/helix_sliding/20250604_r2_hs_lib/final_order'

# Single state designs

In [289]:
r2_ssd_df = pd.read_csv(r2_ssd_seqs_path)
print(f'R2 SSD designs: {r2_ssd_df.shape[0]}')
print(f'R2 SSD unique backbones: {r2_ssd_df.bb_id.nunique()}')

r2_ssd_df['design_class'] = r2_ssd_df['state_design'] + '-' + r2_ssd_df['seq_method']

#create truncated sequence without residues C-terminal to alfa tag
r2_ssd_df['final_sequence'] = r2_ssd_df['sequence'].str.split('SRLEEELRRRLTE').str[0] + 'SRLEEELRRRLTE'

#replace N-term residues with 'GA'
r2_ssd_df['final_sequence'] = 'GA' + r2_ssd_df['final_sequence'].str[2:]

#make a shortname column that is the design class plus a number that increases for each row
r2_ssd_df = r2_ssd_df.reset_index()
r2_ssd_df['shortname'] = r2_ssd_df['design_class'] + '-' + r2_ssd_df.index.astype(str)

#filter out sequences in r1_hs lib
#will add these separately as same nt seqs as r1 order

r1_hs_seqs_path = '/Users/stephaniecrilly/Library/CloudStorage/Box-Box/kortemmelab/home/scrilly/helix_sliding/20231212_final_analysis_r1_order/20231221_final_order/20231221_all_design_sets_r1_order.csv'
r1_hs_df = pd.read_csv(r1_hs_seqs_path)
r2_ssd_no_r1_df = r2_ssd_df.query('not sequence.isin(@r1_hs_df.sequence)', engine='python').copy()

#get shortname and sequence column only
r2_ssd_short_df = r2_ssd_no_r1_df[['shortname', 'final_sequence']]

r2_ssd_df.to_csv(f'{outdir}/r2_ssd_designs_final.csv', index=False)

R2 SSD designs: 495
R2 SSD unique backbones: 223


# Multistate designs

In [290]:
####MSD for designable bbs####
r2_msd_df = pd.read_csv(r2_msd_seqs_path)
print(f'R2 MSD for designable bbs designs: {r2_msd_df.sequence.nunique()}')

#make a shortname column that is the design class plus a number that increases for each row
r2_msd_df['design_class'] = r2_msd_df['state_design_min0'] + '-' + r2_msd_df['seq_design_method_min0'].str.lower()

#create truncated sequence without residues C-terminal to alfa tag
r2_msd_df['final_sequence'] = r2_msd_df['sequence'].str.split('SRLEEELRRRLTE').str[0] + 'SRLEEELRRRLTE'

#replace N-term residues with 'GA'
r2_msd_df['final_sequence'] = 'GA' + r2_msd_df['final_sequence'].str[2:]

#make new column for bb id pair
r2_msd_df['msd_bbs_id'] = r2_msd_df['min0_bb_id_min0'] + '_' + r2_msd_df['min2_bb_id_min0']
print(f"Unique bb pairs for R2 MSD F2S scaled up designs: {r2_msd_df['msd_bbs_id'].nunique()}") #may be an overestimate bc of diff naming conventions (65-24=41 at lowest)

####MSD for experimental bbs####
r2_msd_exp_bbs_df = pd.read_csv(r2_msd_exp_bbs_seqs_path)
print(f'R2 MSD for experimental bbs designs: {r2_msd_exp_bbs_df.sequence.nunique()}')   

#create truncated sequence without residues C-terminal to alfa tag
r2_msd_exp_bbs_df['final_sequence'] = r2_msd_exp_bbs_df['sequence'].str.split('SRLEEELRRRLTE').str[0] + 'SRLEEELRRRLTE'

#replace N-term residues with 'GA'
r2_msd_exp_bbs_df['final_sequence'] = 'GA' + r2_msd_exp_bbs_df['final_sequence'].str[2:]

#make a shortname column that is the design class plus a number that increases for each row
r2_msd_exp_bbs_df['design_class'] = r2_msd_exp_bbs_df['state_design_min0'] + '-exp_bbs-' + r2_msd_exp_bbs_df['seq_design_method_min0'].str.lower()
print(f'Unique bb pairs for R2 MSD exp bbs designs: 1')

####MSD F2S scaled up####
r2_msd_scale_up_df = pd.read_csv(r2_msd_scale_up_seqs_path)
print(f'R2 MSD for F2S scaled up designs: {r2_msd_scale_up_df.sequence.nunique()}')

#create truncated sequence without residues C-terminal to alfa tag
r2_msd_scale_up_df['final_sequence'] = r2_msd_scale_up_df['sequence'].str.split('SRLEEELRRRLTE').str[0] + 'SRLEEELRRRLTE'

#replace N-term residues with 'GA'
r2_msd_scale_up_df['final_sequence'] = 'GA' + r2_msd_scale_up_df['final_sequence'].str[2:]

#make a shortname column that is the design class plus a number that increases for each row
r2_msd_scale_up_df['design_class'] = r2_msd_scale_up_df['state_design_min0'] + '-scale_up-' + r2_msd_scale_up_df['seq_design_method_min0'].str.lower()
print(f"Unique bb pairs for R2 MSD F2S scaled up designs: {r2_msd_scale_up_df['msd_bbs_id_min0'].nunique()}")

#full dfs to concat
r2_msd_all_dfs_to_concat = [r2_msd_df, r2_msd_exp_bbs_df, r2_msd_scale_up_df]
r2_msd_all_df = pd.concat(r2_msd_all_dfs_to_concat, ignore_index=True)

#make a shortname column that is the design class plus a number that increases for each row
r2_msd_all_df = r2_msd_all_df.reset_index()
r2_msd_all_df['shortname'] = r2_msd_all_df['design_class'] + '-' + r2_msd_all_df.index.astype(str)

#get shortname and sequence column only
r2_msd_all_short_df = r2_msd_all_df[['shortname', 'final_sequence']]

r2_msd_all_df.to_csv(f'{outdir}/r2_msd_designs_final.csv', index=False)


R2 MSD for designable bbs designs: 121
Unique bb pairs for R2 MSD F2S scaled up designs: 65
R2 MSD for experimental bbs designs: 153
Unique bb pairs for R2 MSD exp bbs designs: 1
R2 MSD for F2S scaled up designs: 326
Unique bb pairs for R2 MSD F2S scaled up designs: 58


# Multistate predicted as only one state

In [291]:

#take MSD sequences that are low RMSD to only one state, refilter for plddt > 90 and see how many
r2_msd_min0_path = '/Users/stephaniecrilly/Library/CloudStorage/Box-Box/kortemmelab/home/scrilly/helix_sliding/20250604_r2_hs_lib/metric_files/msd/msd_designs_passing_min0_only.csv'
r2_msd_min2_path = '/Users/stephaniecrilly/Library/CloudStorage/Box-Box/kortemmelab/home/scrilly/helix_sliding/20250604_r2_hs_lib/metric_files/msd/msd_designs_passing_min2_only.csv'

####pred only min0####
r2_msd_min0_df = pd.read_csv(r2_msd_min0_path)
r2_msd_min0_df = r2_msd_min0_df.query('avg_plddt_no_loop > 90  & min0_all_rmsd_no_loop < 1.0 & avg_pae_no_loop < 5', engine='python').copy()
print(f'Number of MSD designs predicted to min0 passing stringent filters: {r2_msd_min0_df.sequence.nunique()}')
r2_msd_min0_df_subset = r2_msd_min0_df.sample(n=175, axis=0, random_state=42)

#create truncated sequence without residues C-terminal to alfa tag
r2_msd_min0_df_subset['final_sequence'] = r2_msd_min0_df_subset['sequence'].str.split('SRLEEELRRRLTE').str[0] + 'SRLEEELRRRLTE'

#replace N-term residues with 'GA'
r2_msd_min0_df_subset['final_sequence'] = 'GA' + r2_msd_min0_df_subset['final_sequence'].str[2:]

#make a shortname column that is the design class plus a number that increases for each row
r2_msd_min0_df_subset['design_class'] = r2_msd_min0_df_subset['state_design'] + '-min0-' + r2_msd_min0_df_subset['seq_design_method'].str.lower()

####pred only min2####
r2_msd_min2_df = pd.read_csv(r2_msd_min2_path)
r2_msd_min2_df = r2_msd_min2_df.query('avg_plddt_no_loop > 90  & min2_all_rmsd_no_loop < 1.0  & avg_pae_no_loop < 5', engine='python').copy()
print(f'Number of MSD designs predicted to min2 passing stringent filters: {r2_msd_min2_df.sequence.nunique()}')
r2_msd_min2_df_subset = r2_msd_min2_df.sample(n=175, axis=0, random_state=42)

#create truncated sequence without residues C-terminal to alfa tag
r2_msd_min2_df_subset['final_sequence'] = r2_msd_min2_df_subset['sequence'].str.split('SRLEEELRRRLTE').str[0] + 'SRLEEELRRRLTE'

#replace N-term residues with 'GA'
r2_msd_min2_df_subset['final_sequence'] = 'GA' + r2_msd_min2_df_subset['final_sequence'].str[2:]

#make a shortname column that is the design class plus a number that increases for each row
r2_msd_min2_df_subset['design_class'] = r2_msd_min2_df_subset['state_design'] + '-min2-' + r2_msd_min2_df_subset['seq_design_method'].str.lower()

#full dfs to concat
r2_msd_one_state_dfs_to_concat = [r2_msd_min0_df_subset, r2_msd_min2_df_subset]
r2_msd_one_state_df = pd.concat(r2_msd_one_state_dfs_to_concat, ignore_index=True)

#make a shortname column that is the design class plus a number that increases for each row
r2_msd_one_state_df = r2_msd_one_state_df.reset_index()
r2_msd_one_state_df['shortname'] = r2_msd_one_state_df['design_class'] + '-' + r2_msd_one_state_df.index.astype(str)

#get shortname and sequence column only
r2_msd_one_state_short_df = r2_msd_one_state_df[['shortname', 'final_sequence']]

r2_msd_one_state_df.to_csv(f'{outdir}/r2_msd_one_state_designs_final.csv', index=False)


Number of MSD designs predicted to min0 passing stringent filters: 9446
Number of MSD designs predicted to min2 passing stringent filters: 572


In [292]:
#if decide to turn first two res into GA run the following
# df['final_sequence'] = df['final_sequence'].str[2:]
# df['final_sequence'] = 'GA' + df['final_sequence']

# Control sequences

In [293]:
#make df with additional control seqs from Bram and 375 and point mutants
#TODO: add alfa tag mutants with bm01 ctrl, add more conservative ad to ala
bespoke_ctrls_dict = {'bmp38' : 'GARLAQLKQERAALKQRLAALDQEIAALEWQIQSDPRKKQLYQRLLALISDRFALEQRIAALDQEIAALEAG', 
                 'bmp53' : 'GARLEELIAERLRLVGDLVDLDREIAALEQQIQSDPRKKQLEQRLAALKQERAALEQRIAALDWEIADLDDG', 
                 'bmp31' : 'GARLAQLKQERAALKQRLAALDQEIAALEWQIQSDPRKKQLEQRLAALKQERAALEQRIAALDQEIAALEAG', 
                 'bmp36' : 'GARLAQLKQERAALEQRLAALDQEIAALEWQIQSDPRKKQLEQKLLELQAERLRLIGDIVNLDQQILNLEAG', 
                 'bmp42' : 'GARLAQLKQERAALKQRLDALDQEIAALEWQIQSDPRKKQLYQRLLELIGERLALMGDIFELDVEIAALEAG', 
                 'bmp45' : 'GARLAQLKQERAALKQRLEALDQEIAALEWQIQSDPRKKQLLQRLYELFGERLALFGDIFDLDVEIAALEAG', 
                 'bmp51' : 'GARLAQLKQERAALKQRLDALEQEIAALEWQIQSDPRKKQLQQRLRQLYGERLRLESDIFDLDVEIAALEAG', 
                 'bmp57' : 'GARLAQLKQERAALKQRLEALDQEIAALEWQIQSDPRKKQLLQRLRRLYGERLALFGDIFDLDVEIAALEAG', 
                 'bmp23' : 'GARLAQLKQERAALKQRLAALDQEIAALEWQIQSPDRKKQLEQRLAALKQERAALEQRIAALDQEIAALEAG',
                 '375' : 'EELKRIEEEIAAIEREIARAEEKLKAQESDPRKKGLQAEREKLLEKLAELRKERERLSRLEEELRRRLTELRRRLE',
                 '375-GA' : 'GALKRIEEEIAAIEREIARAEEKLKAQESDPRKKGLQAEREKLLEKLAELRKERERLSRLEEELRRRLTELRRRLE',
                 '375-R74W' : 'EELKRIEEEIAAIEREIARAEEKLKAQESDPRKKGLQAEREKLLEKLAELRKERERLSRLEEELRRRLTELRRWLE',
                 '375-L75W' : 'EELKRIEEEIAAIEREIARAEEKLKAQESDPRKKGLQAEREKLLEKLAELRKERERLSRLEEELRRRLTELRRRWE',
                 '375-77W' : 'EELKRIEEEIAAIEREIARAEEKLKAQESDPRKKGLQAEREKLLEKLAELRKERERLSRLEEELRRRLTELRRRLEW',
                 '375-77W-GA' : 'GALKRIEEEIAAIEREIARAEEKLKAQESDPRKKGLQAEREKLLEKLAELRKERERLSRLEEELRRRLTELRRRLEW',
                 '375-QatF' : 'QELKRIEQEIAAIEQEIARAEQKLKAQESDPRKKGLQAEREKLLEKLAELRKERERLSRLEEELRRRLTELRRRLEW',
                 '375-QatF-cdel' : 'QELKRIEQEIAAIEQEIARAEQKLKAQESDPRKKGLQAEREKLLEKLAELRKERERLSRLEEELRRRLTE',
                 '375-ad-to-A' : 'EEAKRAEEEAAAAEREAARAEEKAKAQESDPRKKGLQAEAEKALEKAAEARKEAERASRLEEELRRRLTELRRRLE',
                 '375-ad-to-A-cdel' : 'EEAKRAEEEAAAAEREAARAEEKAKAQESDPRKKGLQAEAEKALEKAAEARKEAERASRLEEELRRRLTE', 
                 '375-cdel' : 'EELKRIEEEIAAIEREIARAEEKLKAQESDPRKKGLQAEREKLLEKLAELRKERERLSRLEEELRRRLTE', 
                 '375-cdel-GA' : 'GALKRIEEEIAAIEREIARAEEKLKAQESDPRKKGLQAEREKLLEKLAELRKERERLSRLEEELRRRLTE',
                 '375-G53Q' : 'EELKRIEEEIAAIEREIARAEEKLKAQESDPRKKQLQAEREKLLEKLAELRKERERLSRLEEELRRRLTELRRRLE', 
                 '375-G53Q-GA' : 'GALKRIEEEIAAIEREIARAEEKLKAQESDPRKKQLQAEREKLLEKLAELRKERERLSRLEEELRRRLTELRRRLE', 
                 '375-G35Q-cdel' : 'EELKRIEEEIAAIEREIARAEEKLKAQESDPRKKQLQAEREKLLEKLAELRKERERLSRLEEELRRRLTE', 
                 '375-E14Q' : 'EELKRIEEEIAAIQREIARAEEKLKAQESDPRKKGLQAEREKLLEKLAELRKERERLSRLEEELRRRLTELRRRLE', 
                 '375-E14Q-GA' : 'GALKRIEEEIAAIQREIARAEEKLKAQESDPRKKGLQAEREKLLEKLAELRKERERLSRLEEELRRRLTELRRRLE',
                 '375-E14Q-cdel' : 'EELKRIEEEIAAIQREIARAEEKLKAQESDPRKKGLQAEREKLLEKLAELRKERERLSRLEEELRRRLTE', 
                 '375-E14Q-G35Q' : 'EELKRIEEEIAAIQREIARAEEKLKAQESDPRKKQLQAEREKLLEKLAELRKERERLSRLEEELRRRLTELRRRLE',
                 '375-E14Q-G35Q-cdel' : 'EELKRIEEEIAAIQREIARAEEKLKAQESDPRKKQLQAEREKLLEKLAELRKERERLSRLEEELRRRLTE',
                 '375-G35A' : 'EELKRIEEEIAAIEREIARAEEKLKAQESDPRKKALQAEREKLLEKLAELRKERERLSRLEEELRRRLTELRRRLE', 
                 '375-G35A-GA' : 'GALKRIEEEIAAIEREIARAEEKLKAQESDPRKKALQAEREKLLEKLAELRKERERLSRLEEELRRRLTELRRRLE',
                 '375-G35A-cdel' : 'EELKRIEEEIAAIEREIARAEEKLKAQESDPRKKALQAEREKLLEKLAELRKERERLSRLEEELRRRLTE', 
                 '375-L39P' : 'EELKRIEEEIAAIEREIARAEEKLKAQESDPRKKGPQAEREKLLEKLAELRKERERLSRLEEELRRRLTELRRRLE',
                 '375-L39P-GA' : 'GALKRIEEEIAAIEREIARAEEKLKAQESDPRKKGPQAEREKLLEKLAELRKERERLSRLEEELRRRLTELRRRLE',
                 '375-L39P-cdel' : 'EELKRIEEEIAAIEREIARAEEKLKAQESDPRKKGPQAEREKLLEKLAELRKERERLSRLEEELRRRLTE',
                 '375-L36K' : 'EELKRIEEEIAAIEREIARAEEKLKAQESDPRKKGKQAEREKLLEKLAELRKERERLSRLEEELRRRLTELRRRLE',
                 '375-L36K-GA' : 'GALKRIEEEIAAIEREIARAEEKLKAQESDPRKKGKQAEREKLLEKLAELRKERERLSRLEEELRRRLTELRRRLE',
                 '375-L36K-cdel' : 'EELKRIEEEIAAIEREIARAEEKLKAQESDPRKKGKQAEREKLLEKLAELRKERERLSRLEEELRRRLTE',
                 '375-L36Q' : 'EELKRIEEEIAAIEREIARAEEKLKAQESDPRKKGQQAEREKLLEKLAELRKERERLSRLEEELRRRLTELRRRLE',
                 '375-L36Q-GA' : 'GALKRIEEEIAAIEREIARAEEKLKAQESDPRKKGQQAEREKLLEKLAELRKERERLSRLEEELRRRLTELRRRLE',
                 '375-L36Q-cdel' : 'EELKRIEEEIAAIEREIARAEEKLKAQESDPRKKGQQAEREKLLEKLAELRKERERLSRLEEELRRRLTE',
                 '375-ALFA-LtoA' : 'EELKRIEEEIAAIEREIARAEEKLKAQESDPRKKGLQAEREKLLEKLAELRKERERLSRAEEEARRRATELRRRLE',
                 '375-ALFA-LtoA-cdel' : 'EELKRIEEEIAAIEREIARAEEKLKAQESDPRKKGLQAEREKLLEKLAELRKERERLSRAEEEARRRATE',
                 'bm01-alfa-t6-wt' : 'SRLARRRAALKQRIAALKQRRAALKWQIQSDPRKKQLEQELAALDQEIAAAEQELAALDSRLEEELRRRLTE',
                 'bm01-alfa-t6-L1toA' : 'SRLARRRAALKQRIAALKQRRAALKWQIQSDPRKKQLEQELAALDQEIAAAEQELAALDSRAEEELRRRLTE',
                 'bm01-alfa-t6-L2toA' : 'SRLARRRAALKQRIAALKQRRAALKWQIQSDPRKKQLEQELAALDQEIAAAEQELAALDSRLEEEARRRLTE',
                 'bm01-alfa-t6-L3toA' : 'SRLARRRAALKQRIAALKQRRAALKWQIQSDPRKKQLEQELAALDQEIAAAEQELAALDSRLEEELRRRATE',
                 'bm01-alfa-t6-LtoA' : 'SRLARRRAALKQRIAALKQRRAALKWQIQSDPRKKQLEQELAALDQEIAAAEQELAALDSRAEEEARRRATE'}

#make df with columns 'Name and 'sequence' from dict
bespoke_ctrls_df = pd.DataFrame.from_dict(bespoke_ctrls_dict, orient='index', columns=['Sequence']).reset_index().rename(columns={'index': 'Name'})
bespoke_ctrls_df.shape

bespoke_ctrls_df.to_csv(f'{outdir}/ctrl_seqs_final.csv', index=False)

In [294]:
#control seq functions

####Heptad a and d to Ala####
def heptad_to_upper(input_string, split_char = 'r '):
    split_1 = input_string.split(split_char)[0]
    split_2 = input_string.split(split_char)[1]
    split_2 = split_2.upper()
    
    new_string = split_1+split_char+split_2

    return new_string


def heptad_a_d_to_ala(reg_string, seq_string):

    og_seq = re.split(r'\s+', seq_string)[1]
    
    for index, char in enumerate(reg_string):
        if char.isupper():
            if char in ['A', 'D']:
                seq_string = seq_string[:index] + 'A' + seq_string[index+1:]
            else:
                continue
    
    new_seq = re.split(r'\s+', seq_string)[1]

    return og_seq, new_seq


def a_d_to_ala(input_df, state='ssd'):

    new_seqs_dict = {}

    if state == 'ssd':
        h1_reg_colname = 'h1_reg'
        h2_reg_colname = 'h2_reg'

        h1_seq_colname = 'h1_seq'
        h2_seq_colname = 'h2_seq'
    elif state == 'msd':
        h1_reg_colname = 'h1_reg_min2'
        h2_reg_colname = 'h2_reg_min2'

        h1_seq_colname = 'h1_seq_min2'
        h2_seq_colname = 'h2_seq_min2'

    for index, row in input_df.iterrows():

        h1_reg = row[h1_reg_colname]
        new_h1_reg = heptad_to_upper(h1_reg)

        h1_seq = row[h1_seq_colname]

        h1_og, h1_new = heptad_a_d_to_ala(new_h1_reg, h1_seq)

        h2_reg = row[h2_reg_colname]
        new_h2_reg = heptad_to_upper(h2_reg)

        h2_seq = row[h2_seq_colname]

        h2_og, h2_new = heptad_a_d_to_ala(new_h2_reg, h2_seq)

        #replace with ala seq in sequence
        sequence = row['final_sequence_for_ad']
        new_sequence = sequence.replace(h1_og, h1_new)
        new_sequence = new_sequence.replace(h2_og, h2_new)

        new_seqs_dict[sequence] = new_sequence

    new_df = input_df.copy()
    new_df['a_d_to_ala_sequence'] = new_df['final_sequence_for_ad'].map(new_seqs_dict)

    return new_df

####

####SCRAMBLE CONTROLS####
def full_scrambler(sequence_list):
    '''function for full scrambles use random.sample
    Takes a list of original design sequences
    Outputs a dictionary with original seq and fully scrambled seq as key:value pair'''

    full_scramble_dict = {}
    for seq in sequence_list:
        residues = list(seq)
        sc_residues = random.sample(residues, k=len(residues))
        sc_seq = ''.join(sc_residues)
        full_scramble_dict[seq] = sc_seq
    return full_scramble_dict

def patterned_scrambler(sequence_list):
    '''Function for creating patterned scrambled designs
    In which hydrophobic or polar residues are shuffled to different positions with the same h or p character
    prolines and glycines are kept in original positions
    based on criteria described in in Rocklin, et al. 2017
    Takes a list of original design sequences
    Outputs a data frame with the original sequence, pattern of H/P/Pro/Gly, and the patterned scramble sequence'''

    patterned_scramble_dict = {}
    #Rocklin term these polar if they occur in a helix: D, E, H, K, N, Q, R, S, T, or Y
    #A, F, I, L, M, V, W, and Y used to count number of hydrophobic aa
    #I just used hydrophobic and polar definitions from default resfile categories
    hydrophobics_list = ['A', 'F', 'I', 'L', 'M', 'V', 'W', 'Y']
    polar_list = ['D', 'E', 'H', 'K', 'N', 'Q', 'R', 'S', 'T']
    seq_pattern_dict = {}

    for seq in sequence_list:
        residues = list(seq)
        patterned_res = []
        hydrophobic_residues = []
        polar_residues = []
        patterned_scramble_res = []
        for res in residues:
            if res == 'P':
                patterned_res.append('Pro')

            elif res == 'G':
                patterned_res.append('G')

            elif res in hydrophobics_list:
                patterned_res.append('H')
                hydrophobic_residues.append(res)

            elif res in polar_list:
                patterned_res.append('P')
                polar_residues.append(res)

            else:
                print('Cys found!')

        seq_pattern = ''.join(patterned_res)
        seq_pattern_dict[seq] = seq_pattern

        for res in patterned_res:

            if res == 'Pro':
                patterned_scramble_res.append('P')
            elif res == 'G':
                patterned_scramble_res.append('G')
            elif res == 'H':
                l = random.randint(0, len(hydrophobic_residues)-1)
                r = hydrophobic_residues.pop(l)
                patterned_scramble_res.append(r)
            elif res == 'P':
                l = random.randint(0, len(polar_residues)-1)
                r = polar_residues.pop(l)
                patterned_scramble_res.append(r)
        pattern_scramble_seq = ''.join(patterned_scramble_res)
        patterned_scramble_dict[seq] = pattern_scramble_seq

    df_name_pattern = pd.DataFrame.from_dict(seq_pattern_dict, orient='index', columns=['Seq HP pattern'])
    df_name_pattern.index.name = 'Insert protein sequence'
    df_name_pattern.reset_index(inplace=True)

    df_name_pattern_sc = pd.DataFrame.from_dict(patterned_scramble_dict, orient='index', columns=['patterned_scramble_sequence'])
    df_name_pattern_sc.index.name = 'Insert protein sequence'
    df_name_pattern_sc.reset_index(inplace=True)

    df = pd.merge(df_name_pattern, df_name_pattern_sc, on='Insert protein sequence')

    return df

####

In [295]:
#randomly sample 37 from each set to create scrambles

scramble_seq_dfs = []

#for ssd select min2 only so alfa tag not mutated
r2_ssd_mpnn_scrambles_df = r2_ssd_df.query("seq_method == 'mpnn' & min_condition == 'min2'").sample(n=32, axis=0, random_state=42)
r2_ssd_f2s_scrambles_df = r2_ssd_df.query("seq_method == 'f2s' & min_condition == 'min2'").sample(n=32, axis=0, random_state=42)
ssd_scrambles_df = pd.concat([r2_ssd_mpnn_scrambles_df, r2_ssd_f2s_scrambles_df], ignore_index=True)

#make second final_sequence column that will match the h1 and h2 helices without N-term GA
ssd_scrambles_df['final_sequence_for_ad'] = ssd_scrambles_df['sequence'].str.split('SRLEEELRRRLTE').str[0] + 'SRLEEELRRRLTE'

#make ssd ad to ala scrambles
ssd_scrambles_df = a_d_to_ala(ssd_scrambles_df, state='ssd')

#replace N-term residues with 'GA'
ssd_scrambles_df['a_d_to_ala_sequence'] = 'GA' + ssd_scrambles_df['a_d_to_ala_sequence'].str[2:]

#update name
ssd_scrambles_df['shortname'] = ssd_scrambles_df['shortname'] + '_ad_scramble'

ssd_scrambles_df_subset = ssd_scrambles_df[['shortname', 'a_d_to_ala_sequence']]
ssd_scrambles_df_subset = ssd_scrambles_df_subset.rename(columns={'shortname':'Name', 'a_d_to_ala_sequence': 'Sequence'})
scramble_seq_dfs.append(ssd_scrambles_df_subset)

r2_msd_mpnn_scrambles_df = r2_msd_all_df.query("seq_design_method_min0 == 'MPNN'").sample(n=32, axis=0, random_state=42)
r2_msd_f2s_scrambles_df = r2_msd_all_df.query("seq_design_method_min0 == 'f2s_os'").sample(n=32, axis=0, random_state=42)
msd_scrambles_df = pd.concat([r2_msd_mpnn_scrambles_df, r2_msd_f2s_scrambles_df], ignore_index=True)

#make second final_sequence column that will match the h1 and h2 helices without N-term GA
msd_scrambles_df['final_sequence_for_ad'] = msd_scrambles_df['sequence'].str.split('SRLEEELRRRLTE').str[0] + 'SRLEEELRRRLTE'

#make msd ad to ala scrambles
msd_scrambles_df = a_d_to_ala(msd_scrambles_df, state='msd')

#replace N-term residues with 'GA'
msd_scrambles_df['a_d_to_ala_sequence'] = 'GA' + msd_scrambles_df['a_d_to_ala_sequence'].str[2:]

#update name
msd_scrambles_df['shortname'] = msd_scrambles_df['shortname'] + '_ad_scramble'

msd_scrambles_df_subset = msd_scrambles_df[['shortname', 'a_d_to_ala_sequence']]
msd_scrambles_df_subset = msd_scrambles_df_subset.rename(columns={'shortname':'Name', 'a_d_to_ala_sequence': 'Sequence'})
scramble_seq_dfs.append(msd_scrambles_df_subset)

#combine
scramble_dfs_to_concat = [ssd_scrambles_df, msd_scrambles_df]
scrambles_df = pd.concat(scramble_dfs_to_concat, ignore_index=True)

#for full and patterned scrambles, delete the alfa tag first, scramble, then add it back
scrambles_df['duplicated_sequence'] = scrambles_df['final_sequence'].str.split('SRLEEELRRRLTE').str[0]

#full scrambles
full_scramble_dict = full_scrambler(scrambles_df['duplicated_sequence'].tolist())
scrambles_df['full_scramble_sequence'] = scrambles_df['duplicated_sequence'].map(full_scramble_dict)
scrambles_df['full_scramble_sequence'] = scrambles_df['full_scramble_sequence'] + 'SRLEEELRRRLTE'

#update name
scrambles_df['shortname_full'] = scrambles_df['shortname'] + '_full_scramble'
full_scrambles_df_subset = scrambles_df[['shortname_full', 'full_scramble_sequence']]
full_scrambles_df_subset = full_scrambles_df_subset.rename(columns={'shortname_full': 'Name', 'full_scramble_sequence': 'Sequence'})
scramble_seq_dfs.append(full_scrambles_df_subset)

#patterned scrambles
patterned_scramble_df = patterned_scrambler(scrambles_df['duplicated_sequence'].tolist())
scrambles_df = pd.merge(scrambles_df, patterned_scramble_df, left_on='duplicated_sequence', right_on='Insert protein sequence', how='left')
scrambles_df['patterned_scramble_sequence'] = scrambles_df['patterned_scramble_sequence'] + 'SRLEEELRRRLTE'

#update name
scrambles_df['shortname_pat'] = scrambles_df['shortname'] + '_patterned_scramble'
pat_scrambles_df_subset = scrambles_df[['shortname_pat', 'patterned_scramble_sequence']]
pat_scrambles_df_subset = pat_scrambles_df_subset.rename(columns={'shortname_pat': 'Name', 'patterned_scramble_sequence': 'Sequence'})
scramble_seq_dfs.append(pat_scrambles_df_subset)

all_scrambles_df = pd.concat(scramble_seq_dfs, ignore_index=True)
print(f'Number of scrambles: {all_scrambles_df.shape[0]}')

all_scrambles_df.to_csv(f'{outdir}/scramble_seqs_final.csv', index=False)

Number of scrambles: 384


In [296]:
#combine all dfs
r2_ssd_short_df = r2_ssd_short_df.rename(columns={'shortname': 'Name', 'final_sequence': 'Sequence'})
r2_msd_all_short_df = r2_msd_all_short_df.rename(columns={'shortname': 'Name', 'final_sequence': 'Sequence'})
r2_msd_one_state_short_df = r2_msd_one_state_short_df.rename(columns={'shortname': 'Name', 'final_sequence': 'Sequence'})

final_dfs_to_concat = [r2_ssd_short_df, r2_msd_all_short_df, r2_msd_one_state_short_df, all_scrambles_df, bespoke_ctrls_df]
final_df = pd.concat(final_dfs_to_concat, ignore_index=True)
print(f'Final number of sequences: {final_df.shape[0]}')
print(final_df.Name.nunique())
print(final_df.Sequence.nunique())

final_df.to_csv(f'{outdir}/r2_hs_all_protein_seqs_final.csv', index=False)

Final number of sequences: 1813
1813
1813


In [298]:
#load in r1-hs control sequences (nucleotide)
r1_hs_nt_seqs_path = '/Users/stephaniecrilly/Library/CloudStorage/Box-Box/kortemmelab/home/scrilly/helix_sliding/20231212_final_analysis_r1_order/20231221_final_order/20231222_helix_ssd_r1_opt_seqs.csv'
r1_hs_nt_seqs_df = pd.read_csv(r1_hs_nt_seqs_path)

#filter out bm01 seqs
r1_hs_nt_seqs_df = r1_hs_nt_seqs_df.query("not Name.str.startswith('bm01')", engine='python').copy()

#make truncated sequence
leading_bases = 'agcggaggcggagggtcggcttcgcatatg'
trailing_bases = 'ctcgagggtggaggttccgaacaacagctt'

r1_hs_nt_52 = r1_hs_nt_seqs_df.query("Name.str.contains('52')", engine='python').copy()
r1_hs_nt_52['Sequence'] = r1_hs_nt_52['Sequence'].str.split(leading_bases).str[1]
r1_hs_nt_52['Sequence'] = r1_hs_nt_52['Sequence'].str.split(trailing_bases).str[0]
r1_hs_nt_52['Sequence'] = r1_hs_nt_52['Sequence'].str[:-18]
r1_hs_nt_52['Sequence'] = leading_bases + r1_hs_nt_52['Sequence'] + trailing_bases
r1_hs_nt_52['Name'] = r1_hs_nt_52['Name'] + '_r1_trunc'

r1_hs_nt_53 = r1_hs_nt_seqs_df.query("Name.str.contains('53')", engine='python').copy()
r1_hs_nt_53['Sequence'] = r1_hs_nt_53['Sequence'].str.split(leading_bases).str[1]
r1_hs_nt_53['Sequence'] = r1_hs_nt_53['Sequence'].str.split(trailing_bases).str[0]
r1_hs_nt_53['Sequence'] = r1_hs_nt_53['Sequence'].str[:-15]
r1_hs_nt_53['Sequence'] = leading_bases + r1_hs_nt_53['Sequence'] + trailing_bases
r1_hs_nt_53['Name'] = r1_hs_nt_53['Name'] + '_r1_trunc'

r1_hs_nt_seqs_df['Name'] = r1_hs_nt_seqs_df['Name'] + '_r1'

r1_hs_ctrl_seqs_df = pd.concat([r1_hs_nt_seqs_df, r1_hs_nt_52, r1_hs_nt_53], ignore_index=True)
print(r1_hs_ctrl_seqs_df.shape)   

r1_hs_ctrl_seqs_df.to_csv(f'{outdir}/r2_hs_r1_ctrl_nt_seqs_final.csv', index=False)

(178, 3)
